In [ ]:
# 필요한 라이브러리 임포트
import os                             # 파일 및 디렉토리 작업을 위한 모듈
import torch                          # PyTorch 딥러닝 프레임워크
import torch.nn as nn                 # 신경망 관련 모듈
import torch.optim as optim           # 최적화 알고리즘 모듈
from PIL import Image                 # 이미지 처리 모듈 (Pillow)
import shutil                         # 파일 및 폴더 복사/삭제 등을 위한 고급 파일 처리 모듈
from torchvision import transforms, models  # 이미지 변환 및 사전학습 모델
from torch.utils.data import Dataset, DataLoader  # 커스텀 데이터셋 및 배치 로더
import matplotlib.pyplot as plt       # 그래프 시각화
from IPython.display import clear_output  # Jupyter Notebook에서 출력 갱신용

# 디바이스 설정: GPU가 사용 가능한 경우 'cuda', 아니면 'cpu'를 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 삭제할 폴더 경로 지정
folder_path = "checkpoints/train/weights"

# 해당 폴더가 존재하는지 확인
if os.path.exists(folder_path):
    # 폴더 안의 모든 파일과 하위 폴더를 순회
    for filename in os.listdir(folder_path):
        # 각 파일 또는 폴더의 전체 경로를 생성
        file_path = os.path.join(folder_path, filename)
        try:
            # 해당 경로가 '파일' 또는 '심볼릭 링크'인 경우 삭제
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # 파일 또는 링크 삭제
                print(f"파일 '{file_path}'이 삭제되었습니다.")
            # 해당 경로가 '디렉토리'인 경우 재귀적으로 삭제
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # 폴더 전체 삭제
                print(f"폴더 '{file_path}'이 삭제되었습니다.")
        except Exception as e:
            # 삭제 중 오류가 발생한 경우 에러 메시지를 출력
            print(f"'{file_path}'를 삭제하는 중 에러가 발생했습니다: {e}")
else:
    # 폴더가 존재하지 않는 경우 안내 메시지 출력
    print(f"폴더 '{folder_path}'가 존재하지 않습니다.")


In [ ]:
# 하이퍼파라미터 설정
EPOCH = 20        # 총 학습 에포크 수
pre_epoch = 0      # 시작 에포크 (재학습 시 사용)
BATCH_SIZE = 16    # 배치 크기
LR = 0.01          # 학습률
TRAIN_DATA_PERCENT = 0.8  # 훈련 데이터 비율

In [ ]:
# PyTorch의 Dataset 클래스를 상속하여 CustomDataset 정의
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir                    # 데이터셋의 루트 폴더 경로
        self.transform = transform                  # 이미지에 적용할 변환(transform) 함수
        # 클래스 이름 리스트를 오름차순으로 정렬하여 저장
        self.classes = sorted([
            d for d in os.listdir(root_dir)
            if os.path.isdir(os.path.join(root_dir, d))  # 폴더만 추출
        ])
        # 클래스 이름을 숫자 인덱스로 매핑 (예: {'cat': 0, 'dog': 1})
        self.class_to_idx = {
            cls_name: idx for idx, cls_name in enumerate(self.classes)
        }
        # 모든 이미지 경로와 라벨을 불러오는 함수 호출
        self.data = self._load_data()

    def _load_data(self):
        data = []
        for cls in self.classes:
            class_path = os.path.join(self.root_dir, cls)  # 클래스별 디렉토리 경로
            for img_name in os.listdir(class_path):        # 클래스 폴더 안의 모든 이미지 파일
                img_path = os.path.join(class_path, img_name)
                label = self.class_to_idx[cls]              # 클래스 이름을 숫자로 매핑
                data.append((img_path, label))              # (이미지 경로, 라벨) 튜플 저장
        return data

    def __len__(self):
        return len(self.data)  # 전체 데이터 개수 반환

    def __getitem__(self, idx):
        img_path, label = self.data[idx]                    # 인덱스에 해당하는 이미지 경로와 라벨
        image = Image.open(img_path).convert('RGB')         # 이미지 파일 열고 RGB로 변환
        if self.transform:                                  # transform이 설정되어 있으면 적용
            image = self.transform(image)
        return image, label                                 # 이미지와 라벨 반환


# 데이터 전처리 설정: 이미지를 224x224 크기로 리사이즈하고 텐서로 변환
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # 입력 크기를 사전학습 모델 크기에 맞춤
    transforms.ToTensor()            # [0, 1] 범위의 Tensor로 변환
])

# 전체 데이터셋 로드 (예: 'data' 폴더 구조는 data/class1/*.jpg, data/class2/*.jpg ...)
dataset = CustomDataset(root_dir="data", transform=transform)

# 전체 데이터셋에서 훈련/테스트 비율 설정
# 예: TRAIN_DATA_PERCENT = 0.8이라면 훈련 80%, 테스트 20%
train_size = int(TRAIN_DATA_PERCENT * len(dataset))
test_size = len(dataset) - train_size

# 데이터셋을 무작위로 훈련 세트와 테스트 세트로 분할
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# DataLoader는 데이터를 배치 단위로 불러오고, 멀티 스레딩(num_workers)을 활용
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# 훈련/테스트 데이터셋의 크기 출력
print(f"훈련 데이터 수: {len(train_dataset)}, 테스트 데이터 수: {len(test_dataset)}")


In [ ]:
# 클래스 수 확인
classes = dataset.classes        # 커스텀 데이터셋에서 클래스 이름 목록 추출
NUM_CLASSES = len(classes)       # 전체 클래스 수 계산 (예: 3개 클래스면 3)

# 사전 학습된 ResNet-18 모델 불러오기
net = models.resnet50()          # torchvision에서 ResNet-18 모델 가져오기 (기본적으로 pretrained=False)

# 모델의 마지막 fully connected layer 입력 특징 수 확인
num_ftrs = net.fc.in_features    # 기존 FC layer에 들어가는 입력 뉴런 수 (예: 512)

# 마지막 FC layer를 현재 문제에 맞게 커스터마이징
# 원래는 ImageNet 기준 1000개 클래스였지만, 우리는 NUM_CLASSES에 맞게 조정
net.fc = nn.Linear(num_ftrs, NUM_CLASSES)

# 모델을 CUDA(GPU) 또는 CPU로 이동 (앞에서 설정한 device에 따라)
net = net.to(device)


In [ ]:
# 손실 함수 및 옵티마이저 정의

# 다중 클래스 분류에서 자주 사용되는 손실 함수: CrossEntropyLoss
# softmax와 negative log likelihood를 함께 수행함
criterion = nn.CrossEntropyLoss()

# 확률적 경사 하강법(SGD)을 사용한 옵티마이저 설정
# - net.parameters(): 학습할 모델의 모든 파라미터
# - lr: 학습률 (learning rate), 너무 크면 발산하고, 너무 작으면 수렴이 느림
# - momentum: 관성 항으로, 이전의 업데이트 방향을 일부 반영하여 최적화의 속도를 높임
# - weight_decay: L2 정규화 계수로 과적합을 방지함
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)


In [ ]:
# 훈련 및 테스트 결과를 저장할 리스트를 초기화합니다.
train_losses = []       # 각 에폭(epoch)마다의 평균 훈련 손실(loss)을 저장할 리스트
train_accuracies = []   # 각 에폭마다의 훈련 정확도(accuracy)를 저장할 리스트
test_accuracies = []    # 각 에폭마다의 테스트 정확도를 저장할 리스트

# 훈련 루프를 시작합니다. 전체 훈련은 지정된 에폭 수(EPOCH)만큼 반복됩니다.
for epoch in range(EPOCH):
    print(f'\n🔁 에폭: {epoch + 1}')  # 현재 진행 중인 에폭 번호를 출력
    net.train()  # 모델을 훈련 모드로 설정 (Dropout, BatchNorm 등의 동작 방식이 변경됨)

    sum_loss = 0.0  # 한 에폭 동안의 전체 손실 값을 누적할 변수
    correct = 0     # 정확히 예측한 샘플 개수 초기화
    total = 0       # 전체 샘플 개수 초기화

    # 훈련 데이터셋을 하나씩 반복하며 학습을 수행합니다.
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data  # 데이터로부터 입력 이미지와 정답 레이블을 분리
        inputs, labels = inputs.to(device), labels.to(device)  # GPU 또는 CPU로 데이터 이동

        optimizer.zero_grad()  # 이전 배치에서 계산된 기울기 정보를 초기화

        outputs = net(inputs)              # 모델에 입력값을 넣어 예측값(출력)을 얻음
        loss = criterion(outputs, labels)  # 예측값과 실제값 사이의 손실을 계산
        loss.backward()                    # 손실을 기준으로 역전파 수행 (기울기 계산)
        optimizer.step()                   # 계산된 기울기를 이용하여 모델 파라미터 업데이트

        sum_loss += loss.item()  # 손실 값을 누적하여 평균 계산에 사용
        _, predicted = torch.max(outputs.data, 1)  # 예측된 클래스 중 가장 확률 높은 값 선택
        total += labels.size(0)  # 전체 샘플 개수 누적
        correct += predicted.eq(labels.data).sum().item()  # 정답과 일치한 예측 개수 누적

    # 한 에폭의 평균 손실과 정확도를 리스트에 저장
    train_losses.append(sum_loss / len(train_loader))
    train_accuracies.append(100. * correct / total)

    # 현재까지 학습된 모델을 TorchScript 형식으로 저장 (추론 최적화 및 배포 용이)
    model_scripted = torch.jit.script(net)
    model_scripted.save(os.path.join('checkpoints/train/weights', f'resnet_{epoch + 1}.pt'))

    # 모델 성능 평가를 위해 테스트 데이터셋을 사용하여 테스트를 수행합니다.
    print('🧪 테스트 진행 중...')
    net.eval()  # 모델을 평가 모드로 전환 (Dropout 비활성화 등)
    with torch.no_grad():  # 테스트 시에는 그래디언트를 계산하지 않아 메모리 절약
        correct = 0
        total = 0
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += predicted.eq(labels.data).sum().item()

    test_accuracy = 100. * correct / total  # 테스트 정확도 계산
    test_accuracies.append(test_accuracy)   # 리스트에 기록

    # 훈련 중 실시간으로 손실 및 정확도 그래프를 출력합니다.
    clear_output(wait=True)
    plt.figure(figsize=(10, 5))

    # 🟠 훈련 손실 그래프
    plt.subplot(1, 2, 1)
    plt.plot(range(1, epoch + 2), train_losses, marker='o', label='Train Loss')
    plt.title('Train Loss')
    plt.xlabel('Epoch')  # 에폭 수
    plt.ylabel('Loss')   # 손실 값
    plt.legend()

    # 🔵 훈련 및 테스트 정확도 그래프
    plt.subplot(1, 2, 2)
    plt.plot(range(1, epoch + 2), train_accuracies, marker='o', label='Train Accuracy')
    plt.plot(range(1, epoch + 2), test_accuracies, marker='o', label='Test Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')  # 에폭 수
    plt.ylabel('%')      # 정확도 비율
    plt.legend()

    plt.tight_layout()  # 그래프 간격을 자동 조정
    plt.show()

    # 현재 에폭의 훈련 및 테스트 결과 출력
    print(f'[에폭 {epoch + 1}] 훈련 손실: {train_losses[-1]:.3f} | '
          f'훈련 정확도: {train_accuracies[-1]:.2f}% | '
          f'테스트 정확도: {test_accuracies[-1]:.2f}%')

# 모든 훈련 에폭이 완료되었음을 알리는 메시지 출력
print(f'\n✅ 훈련이 완료되었습니다. 총 에폭 수: {EPOCH}')
